In [11]:
import numpy as np
from time import sleep
import sys

import tensorflow as tf
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Input, Dense, Activation, TimeDistributed, LSTM, Dropout
from keras.preprocessing import image
from keras import backend as K

from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split

import pylab
from PIL import Image
import imageio
#imageio.plugins.ffmpeg.download()

import scipy
import os

In [2]:
EPOCHS = 20
VIDEO_FOLDER = (os.getcwd() + '/videos/Clips')
BATCH_SIZE = 32
FRAME_SQUARE_DIM = 178
FRAMES_PER_VIDEO = 30
KEEP_PROB = 0.7
TEST_SIZE = 0.2

In [32]:
# Make Frames directories in Truthful and Deceptive folders
if not os.path.exists((VIDEO_FOLDER + '/Truthful/Frames')):
    os.mkdir(VIDEO_FOLDER + '/Truthful/Frames')
if not os.path.exists((VIDEO_FOLDER + '/Deceptive/Frames')):
    os.mkdir(VIDEO_FOLDER + '/Deceptive/Frames')

In [3]:
def resize_and_crop(frame, square_dim):
    # get aspect ratio
    aspect = frame.size[0]/frame.size[1]
    
    # resize to 178 on shortest side and keep original aspect ratio
    frame = frame.resize((int(square_dim*aspect), square_dim))
    
    # crop 178 square from center
    half_the_width = frame.size[0] / 2
    half_the_height = frame.size[1] / 2
    frame = frame.crop(
        (
            half_the_width - (square_dim/2),
            half_the_height - (square_dim/2),
            half_the_width + (square_dim/2),
            half_the_height + (square_dim/2)
        )
    )
    
    return frame

In [4]:
def get_frames_from_videos(folder, frames_per_video, frame_square_dim):
    
    # get one video at a time in the folder
    vid_counter = 0
    for video_file in os.listdir(folder):
        vid_counter += 1
        
        if video_file not in ['Frames']:
            # printing status
            sys.stdout.write('\r')
            sys.stdout.flush()
            sys.stdout.write('Getting frames for video ' + str(vid_counter) + ' of ' + str(len(os.listdir(folder))))
            sys.stdout.flush()
            
            video = (folder + '/' + video_file)
            video = imageio.get_reader(video,  'ffmpeg')

            # get 30 evenly spaced frames per video
            step = video.get_meta_data()['nframes'] / frames_per_video
            frames_to_get = range(1, video.get_meta_data()['nframes'], int(step))
            frames = []
            for i in frames_to_get:
                frames.append(i)

            # only get first 30 frames if there are 31
            frames = frames[:30]

            # Resize, crop, and save each frame in the Frames folder
            ## ex: (os.getcwd + '/videos/Clips/Truthful/Frames') for Truth videos
            for frame in frames:
                this_frame = video.get_data(frame)
                imageio.imwrite("this_frame.jpg", this_frame)
                this_frame = Image.open("this_frame.jpg")
                this_frame = resize_and_crop(this_frame, frame_square_dim)
                this_frame.save((folder + '/Frames/' + video_file + '_0' + str(frame) + '.jpg'))
                
    print('\n')
    print('Complete!')

In [5]:
def get_data(folder, frames_per_video):
    X = []
    y = []

    for vid_type in os.listdir(folder):
        if not vid_type.startswith('.'):
            if vid_type in ['Deceptive']:
                label = 'Deceptive'
            else:
                label = 'Truthful'
            
            video_tensor = []
            frame_counter = 1
            frame_files = os.listdir(folder + '/' + vid_type + '/Frames')
            for frame_file_iter in range(len(frame_files)):
                if frame_file_iter == 0:
                    continue
                
                frame = (folder + '/' + vid_type + '/Frames/' + frame_files[frame_file_iter])

                frame = Image.open(frame)

                frame_tensor = np.asarray(frame.convert('RGB'))
                
                video_tensor.append(frame_tensor)
                
                if frame_counter == frames_per_video:
                    X.append(video_tensor)
                    y.append(label)
                    
                    video_tensor = []
                    frame_counter = 0
                    
                frame_counter += 1
                
    X = np.asarray(X)
    y = np.asarray(y)
    print('Complete!')
    return X,y

In [35]:
# Prepare video clips into a resized and cropped series of frames for each video
get_frames_from_videos((VIDEO_FOLDER + '/Deceptive'), FRAMES_PER_VIDEO, FRAME_SQUARE_DIM)

There are 62 video files.


In [37]:
# Prepare video clips into a resized and cropped series of frames for each video
get_frames_from_videos((VIDEO_FOLDER + '/Truthful'), FRAMES_PER_VIDEO, FRAME_SQUARE_DIM)

Complete!rames for video 61 of 61


In [6]:
# Get video tensors ready for network input
X, y = get_data(VIDEO_FOLDER, FRAMES_PER_VIDEO)

Complete!


In [7]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)

In [8]:
# Encode target labels
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
# NEXT IS TO TEST NN
# AND THEN TRY IMAGE AUGMENTATION

In [14]:
def get_model(frames_per_video, frame_square_dim, keep_prob):
    # Video file placeholder. Shape = (# of frames, frame-width, frame-length, # of frame-channels (i.e. rgb))
    video = Input(shape=(frames_per_video, frame_square_dim, frame_square_dim, 3))

    # TRANSFER LEARNING LAYER
    # Initialize CNN with weights from InceptionV3 trained on Imagenet.
    IncV3 = InceptionV3(input_tensor=video, weights='imagenet', include_top=False, pooling='avg')
    
    # add a global spatial average pooling layer
    #x = IncV3.output
    #x = GlobalAveragePooling2D()(x)
    #x = GlobalAveragePooling2D()(x)

    # TIME DISTRIBUTION LAYER
    # Run each frame through an InceptionV3 CNN layer.
    encoded_frames = TimeDistributed(IncV3)(video)

    # LSTM LAYER
    # Run each frames CNN output through the LSTM layer.
    encoded_vid = LSTM(256)(encoded_frames)

    # ADDITIONAL LAYERS TO TRAIN NEW CLASSES from our new video footage.
    # Add a fully-connected layer with a dropout before predictions.
    x = Dense(1024, activation='relu')(encoded_vid)
    x = Dropout(keep_prob)(x)

    # Add a logistic layer with 2 new classes - "Lying" and "Truth".
    predictions = Dense(2, activation='sigmoid')(x)

    # Throw it all into a Model object.
    model = Model(input=IncV3.input, output=predictions)

    # Freeze convolutional InceptionV3 layers because we want to start with the it's pre-trained weights.
    # This is the "learning" that's being "transferred".
    for layer in IncV3.layers:
        layer.trainable = False

    # Compile the model.
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [15]:
model = get_model(FRAMES_PER_VIDEO, FRAME_SQUARE_DIM, KEEP_PROB)
print(model.summary())

ValueError: Input 0 is incompatible with layer conv2d_189: expected ndim=4, found ndim=5

In [ ]:
# GET BATCHES OR DEFINE GENERATOR

# Train the model on the new video footage.
model.fit_generator(...)